## Applied Data Science Capstone 
#### IBM Data Science Professional Certificate

#### Introduction
[Makati City, Philippines](https://en.wikipedia.org/wiki/Makati) is one of the most densely populated cities in the world ([#14 as of 2016](https://en.wikipedia.org/wiki/List_of_cities_proper_by_population_density)); with a population density of \~24k / km^2, it out-crowds New York City (\~11k / km^2) and Toronto (\~4.5k / km^2). Aside from all those people, it is also the address of the country's Central Business District, making it a very popular place to open establishments of all categories and at all price points.

Establishments in this area normally 'trend': as one establishment (say, a bar) in one area begins to gain popularity and foot traffic, adjacent complementary establishments -- in this example, coffee shops, restaurants, and other bars -- also enjoy an increase in patronage. The buyer behavior here is driven by the difficulty of parking in the city; once patrons have found parking for their vehicles, the consumer preference leans heavily toward carrying out all their plans (eating, having a coffee, some drinks) in establishments that are walking distance from each other.

#### Objective of Analysis
With this mindset of a walking distance ripple effect, the objective of this analysis is to find prospective locations for a new bar, restaurant, or coffee shop establishment, based on currently popular establishments in those three categories. Intersections of an arbitrary 'walking distance' radius would be ranked higher, since they would have a higher expected benefit due to overlapping effect of nearby popular establishments.

##### Conceptual objective / output of analysis:
<a href="https://ibb.co/qJpQKN5"><img src="https://i.ibb.co/9ZW56bt/Presentation1.jpg" alt="Presentation1" border="0"></a>

The business application of the analysis would be the objective recommendation of locations that could be up-and-coming: not popular yet (hence rent and other expenses may not be as high as in the hotspots), but hold promise for becoming popular in the next 18-36 months due to proximity to other popular destinations.

#### Data Collection

In [1]:
# geometric center determined using intersection of smallest radii
# geometric centering used because Google Maps center is close to the border with adjacent city, introducing a lot of data noise
# approximately corresponds to address: 5 Kawayan Rd., Makati 1219 Philippines
mkt_ctr_lat = 14.55219
mkt_ctr_long = 121.034402

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
# querying top 10 restaurants, bars, and cafes (3 separate categories, 30 data points total) around the center, from FourSquare; an additional 10 places were queried, in case some of the results need to be discarded
restaurants_catID = '4d4b7105d754a06374d81259'
coffeeshop_catID = '4bf58dd8d48988d1e0931735,4bf58dd8d48988d16d941735'
nightlife_catID = '4d4b7105d754a06376d81259'

# setting up queries per category

query_restaurants = 'https://api.foursquare.com/v2/venues/explore?v=20190320&client_id={}&client_secret={}&ll={},{}&categoryId={}&limit=20&time=any&sortByPopularity=1'.format(
    Client_ID, Client_Secret, mkt_ctr_lat, mkt_ctr_long, restaurants_catID)

query_cafes = 'https://api.foursquare.com/v2/venues/explore?v=20190320&client_id={}&client_secret={}&ll={},{}&categoryId={}&limit=20&time=any&sortByPopularity=1'.format(
    Client_ID, Client_Secret, mkt_ctr_lat, mkt_ctr_long, coffeeshop_catID)
    
query_bars = 'https://api.foursquare.com/v2/venues/explore?v=20190320&client_id={}&client_secret={}&ll={},{}&categoryId={}&limit=20&time=any&sortByPopularity=1'.format(
    Client_ID, Client_Secret, mkt_ctr_lat, mkt_ctr_long, nightlife_catID)

query_restaurants

'https://api.foursquare.com/v2/venues/explore?v=20190320&client_id=NDQQNS0HYVYR4ZJMLUFFMTHW2GNA5M0T3YZ5TWT3AYZ0HLJI&client_secret=43YDHSN4KYKLALGYLP3P4SZCDDKXXQJS3ZKS2T3KECGSAKDR&ll=14.55219,121.034402&categoryId=4d4b7105d754a06374d81259&limit=20&time=any&sortByPopularity=1'

In [4]:
# calling request and data handlers
import requests
import pandas as pd
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize

In [5]:
# issuing queries
rawresp_restaurants = requests.get(query_restaurants).json()
rawresp_restaurants

{'meta': {'code': 200, 'requestId': '5e844707923935001b3917a7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 989,
  'headerLocation': 'Forbes Park',
  'headerFullLocation': 'Forbes Park, Makati City',
  'headerLocationGranularity': 'neighborhood',
  'query': 'food',
  'totalResults': 52,
  'suggestedBounds': {'ne': {'lat': 14.561188505725507,
    'lng': 121.04394448633288},
   'sw': {'lat': 14.54651785115043, 'lng': 121.02450226255557}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5a664a5126659b35d9511631',
       'name': 'Marugame Udon',
       'location': {'address': '3/F Glorietta 4, Courtyard Dr, Ayala Center',
        'lat': 14.550808888423173,
        'lng': 121.02624

In [6]:
rawresp_cafes = requests.get(query_cafes).json()
rawresp_cafes

{'meta': {'code': 200, 'requestId': '5e84478f47b43d002360af98'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 989,
  'headerLocation': 'Forbes Park',
  'headerFullLocation': 'Forbes Park, Makati City',
  'headerLocationGranularity': 'neighborhood',
  'query': 'coffee shop',
  'totalResults': 75,
  'suggestedBounds': {'ne': {'lat': 14.558362729594323,
    'lng': 121.045674031682},
   'sw': {'lat': 14.546829426818364, 'lng': 121.0239292575401}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '59f192c56f0aa252bb503462',
       'name': 'Tim Hortons',
       'location': {'address': 'G/F Glorietta 4, Courtyard Dr, Ayala Center',
        'crossStreet': 'btwn North & East Dr',
        '

In [7]:
rawresp_bars = requests.get(query_bars).json()
rawresp_bars

{'meta': {'code': 200, 'requestId': '5e844748edbcad001b7e03e5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 989,
  'headerLocation': 'Forbes Park',
  'headerFullLocation': 'Forbes Park, Makati City',
  'headerLocationGranularity': 'neighborhood',
  'query': 'nightlife',
  'totalResults': 12,
  'suggestedBounds': {'ne': {'lat': 14.559190004200634,
    'lng': 121.0418985440974},
   'sw': {'lat': 14.546583952951497, 'lng': 121.02468038998184}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bdda77b0ee3a59311182eb0',
       'name': 'Whistlestop',
       'location': {'address': '28 Jupiter St',
        'crossStreet': 'at Astra St',
        'lat': 14.558617001871125,
        'lng'

In [8]:
results_restaurants = rawresp_restaurants['response']['groups'][0]['items']
results_restaurants2 = json_normalize(results_restaurants)
df_restaurants = results_restaurants2[['venue.name','venue.location.state', 'venue.location.lat', 'venue.location.lng', 'venue.location.address', 'venue.id']]
df_restaurants.head()

,venue.name,venue.location.state,venue.location.lat,venue.location.lng,venue.location.address,venue.id
0,Marugame Udon,Makati City,14.550809,121.026249,"3/F Glorietta 4, Courtyard Dr, Ayala Center",5a664a5126659b35d9511631
1,Tim Ho Wan,Makati City,14.552159,121.025458,G/F Glorietta 3,54647f5d498e9169e65819ce
2,Outback Steakhouse,Makati City,14.550850,121.026515,"Ground Flr, Glorietta 4",4b7b1c2cf964a520f7512fe3
3,McDonald's,Makati City,14.555526,121.034634,"G/F, SM Cyber Two, Sen. Gil Puyat Ave",4c200aed920076b07abac5e9
4,Hokkaido Ramen Santouka,Makati City,14.550445,121.026626,"Ground Flr, Glorietta 4",5125a7dee4b06c4475697e77


In [9]:
results_cafes = rawresp_cafes['response']['groups'][0]['items']
results_cafes2 = json_normalize(results_cafes)
df_cafes = results_cafes2[['venue.name','venue.location.state', 'venue.location.lat', 'venue.location.lng', 'venue.location.address', 'venue.id']]
df_cafes.head()

,venue.name,venue.location.state,venue.location.lat,venue.location.lng,venue.location.address,venue.id
0,Tim Hortons,Makati City,14.551287,121.026474,"G/F Glorietta 4, Courtyard Dr, Ayala Center",59f192c56f0aa252bb503462
1,FRNK Milk Bar,Makati City,14.551634,121.024924,"G/F, Glorietta 3",5ce93982c4df1d002c9e425d
2,Starbucks,Makati City,14.552657,121.024986,"Ground Flr, 6750 Ayala Tower, 6750 Ayala Ave",4dd26268cc3f7f8c122e7e1e
3,Starbucks,Makati City,14.551298,121.026868,"4th Flr, Glorietta 4",4b63d707f964a520c5932ae3
4,Myron's,Makati City,14.551258,121.026636,Courtyard Dr Fl 6,580ff4db38fad54ea66362ee


In [10]:
results_bars = rawresp_bars['response']['groups'][0]['items']
results_bars2 = json_normalize(results_bars)
df_bars = results_bars2[['venue.name','venue.location.state', 'venue.location.lat', 'venue.location.lng', 'venue.location.address', 'venue.id']]
df_bars.head()

,venue.name,venue.location.state,venue.location.lat,venue.location.lng,venue.location.address,venue.id
0,Whistlestop,Makati City,14.558617,121.032837,28 Jupiter St,4bdda77b0ee3a59311182eb0
1,The Fort at BGC,NaN,14.547157,121.041116,NaN,57091609498ebe45cc36186d
2,The Mercato Centrale,Makati City,14.551373,121.026852,Glorietta 4 Park,524fed71498ebbcb661edc49
3,SM Prestige Lounge,Makati City,14.549154,121.026894,4/F SM Makati South Dr,4c1f0a5db306c92858f067b7
4,Moonshine Kitchen,Makati City,14.550781,121.025585,Glorietta 1 4th Floor Roof Deck,5a23b0fb3c858d64d95d0306


In [30]:
# cleanup of unwanted values
# upon inspection (domain knowledge), first 10 entries -- corresponding to top 10 FourSquare recommendations -- are indeed restaurants in Makati City
# only the first 10 entries are retained
df_restaurants_map = df_restaurants.iloc[0:10,0:7]
df_restaurants_map.rename(columns={"venue.name":"venue_name", "venue.location.state":"venue_location_state", "venue.location.lat":"location_lat", "venue.location.lng":"location_lng"}, inplace=True)
df_restaurants_map.to_csv('df_restaurants_map.csv')
df_restaurants_map

,venue_name,venue_location_state,location_lat,location_lng,venue.location.address,venue.id
0,Marugame Udon,Makati City,14.550809,121.026249,"3/F Glorietta 4, Courtyard Dr, Ayala Center",5a664a5126659b35d9511631
1,Tim Ho Wan,Makati City,14.552159,121.025458,G/F Glorietta 3,54647f5d498e9169e65819ce
2,Outback Steakhouse,Makati City,14.550850,121.026515,"Ground Flr, Glorietta 4",4b7b1c2cf964a520f7512fe3
3,McDonald's,Makati City,14.555526,121.034634,"G/F, SM Cyber Two, Sen. Gil Puyat Ave",4c200aed920076b07abac5e9
4,Hokkaido Ramen Santouka,Makati City,14.550445,121.026626,"Ground Flr, Glorietta 4",5125a7dee4b06c4475697e77
5,Italianni's,Makati City,14.550869,121.026045,"2nd Flr, Glorietta 4",4ebf4dcd02d5a63a3eebe153
6,TGI Fridays,Makati City,14.552243,121.025386,"3rd Flr, Glorietta 3",4b5994fff964a5207b8d28e3
7,La Cabrera,Makati City,14.552801,121.025653,G/F 6750 Ayala Ave,53c90694498e29644ea95b99
8,Cibo,Makati City,14.551989,121.025966,"2nd Flr, Glorietta 4",4b6e3b56f964a5202eb32ce3
9,Va Bene Pasta Deli,Makati City,14.547686,121.026627,"2nd Flr, Petron Service Station",4c2f09ea213c2d7fb933305d


In [32]:
# cleanup of unwanted values

# drop NaN values (they correspond to locations in the wrong city)
df_cafes.dropna(axis=0, how='any', subset=['venue.location.state'], inplace=True)
df_cafes.reset_index(drop=True, inplace=True)

# only the first 10 entries are retained for mapping
df_cafes_map = df_cafes.iloc[0:10,0:7]
df_cafes_map.rename(columns={"venue.name":"venue_name", "venue.location.state":"venue_location_state", "venue.location.lat":"location_lat", "venue.location.lng":"location_lng"}, inplace=True)
df_cafes_map.to_csv('df_cafes_map.csv')
df_cafes_map

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,venue_name,venue_location_state,location_lat,location_lng,venue.location.address,venue.id
0,Tim Hortons,Makati City,14.551287,121.026474,"G/F Glorietta 4, Courtyard Dr, Ayala Center",59f192c56f0aa252bb503462
1,FRNK Milk Bar,Makati City,14.551634,121.024924,"G/F, Glorietta 3",5ce93982c4df1d002c9e425d
2,Starbucks,Makati City,14.552657,121.024986,"Ground Flr, 6750 Ayala Tower, 6750 Ayala Ave",4dd26268cc3f7f8c122e7e1e
3,Starbucks,Makati City,14.551298,121.026868,"4th Flr, Glorietta 4",4b63d707f964a520c5932ae3
4,Myron's,Makati City,14.551258,121.026636,Courtyard Dr Fl 6,580ff4db38fad54ea66362ee
5,Bo's Coffee,Makati City,14.551551,121.026874,"Ground Flr, Glorietta 5",4bdc281a63c5c9b6c0912a68
6,Starbucks,Makati City,14.551779,121.026058,"Ground Flr, Marks & Spencer Store, Glorietta 4",4fd73863e4b0b28c87435dd6
7,UCC Park Café,Makati City,14.552756,121.025346,Glorietta 3 Park,4b96fadbf964a5208ff134e3
8,The Pantry,Makati City,14.548744,121.025739,Ayala Centre,55fcdf7d498e81836aa8199b
9,Starbucks,Makati City,14.547354,121.034877,"2/F San Antonio Plaza Arcade, McKinley Rd, For...",4bd110b99854d13a651bf94d


In [13]:
df_bars = results_bars2[['venue.name','venue.location.state', 'venue.location.lat', 'venue.location.lng', 'venue.location.address', 'venue.id']]

In [33]:
# cleanup of unwanted values

# removal (using domain knowledge) of venues that are not bars
df_bars_map = df_bars.drop(df_bars.index[[1,2,3,5]])
df_bars_map.reset_index(drop=True, inplace=True)
df_bars_map.rename(columns={"venue.name":"venue_name", "venue.location.state":"venue_location_state", "venue.location.lat":"location_lat", "venue.location.lng":"location_lng"}, inplace=True)
df_bars_map.to_csv('df_bars_map.csv')
df_bars_map

# since there are only 8 rows remaining, no further need to slice the df

,venue_name,venue_location_state,location_lat,location_lng,venue.location.address,venue.id
0,Whistlestop,Makati City,14.558617,121.032837,28 Jupiter St,4bdda77b0ee3a59311182eb0
1,Moonshine Kitchen,Makati City,14.550781,121.025585,Glorietta 1 4th Floor Roof Deck,5a23b0fb3c858d64d95d0306
2,RJ Bistro,Makati City,14.549038,121.026544,"Basement, Dusit Thani Manila",4d2da13ea40da35d4b789377
3,Lobby Lounge,Makati City,14.548755,121.025920,Dusit Thani Hotel Manila,516253ade4b03667098a61c1
4,ABV Cocktail and Absinthe Bar,Makati City,14.558203,121.033267,22 Jupiter St,53b3dd1a498eeca9e2c9893a
5,The Distillery,Makati City,14.557763,121.033399,"20 Jupiter Street, Bel-Air Village",4b4b3c2af964a520239526e3
6,Backroom,Makati City,14.557847,121.033390,Jupiter St,5478a8a0498e3bbc45e4755c
7,Casa Nami Surf Pub,Makati City,14.558542,121.032729,Jupiter St.,4ca47c91750ca09301802167


In [19]:
# importing mapping libraries
!conda install folium -c conda-forge
import folium
from IPython.display import Image
from IPython.core.display import HTML

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    folium-0.10.1              |             py_0          59 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTAL

In [37]:
mkt_map = folium.Map(location=[mkt_ctr_lat, mkt_ctr_long], zoom_start=14)

# add a red circle to map the geometric center
folium.Circle(
    [mkt_ctr_lat, mkt_ctr_long],
    radius=50,
    color='red',
    popup='Geometric Center',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6).add_to(mkt_map)

mkt_map

In [38]:
# using 1km map radius as our definition of walking distance
walk_radius_meters = 100

# adding restaurant markers and proximity circle
for lat, lng in zip(df_restaurants_map.location_lat, df_restaurants_map.location_lng):
    folium.Circle(
        [lat, lng],
        radius=walk_radius_meters,
        color='blue',
        popup='Restaurant',
        fill = True,
        fill_color='blue',
        fill_opacity=0.1
    ).add_to(mkt_map)

# adding cafes markers and proximity circle
for lat, lng in zip(df_cafes_map.location_lat, df_cafes_map.location_lng):
    folium.Circle(
        [lat, lng],
        radius=walk_radius_meters,
        color='yellow',
        popup='Cafe',
        fill = True,
        fill_color='yellow',
        fill_opacity=0.1
    ).add_to(mkt_map)

# adding bars markers and proximity circle
for lat, lng in zip(df_bars_map.location_lat, df_bars_map.location_lng):
    folium.Circle(
        [lat, lng],
        radius=walk_radius_meters,
        color='orange',
        popup='Bar',
        fill = True,
        fill_color='orange',
        fill_opacity=0.1
    ).add_to(mkt_map)

mkt_map

Looking at the above output, there is promising insight from the bars category, pointing at Jupiter St. However, most of the results of restaurants and cafes categories reoccur within the Glorietta landmark -- a shopping mall.

This doesn't give a lot of the desired insight, so one more iteration is to be done, this time disqualifying results within the bounds of the landmark.

Bounds of Glorietta landmark:

West-most point: Makati Ave cor Palm Dr -- lat = 14.551756, long = 121.022917

East-most point: Ayala Ave cor East St -- lat = 14.551689, long = 121.027825

North-most point: Makati Ave cor Ayala Ave -- lat = 14.554477, long = 121.024440

South-most point: Palm Dr cor East St -- lat = 14.549851, long = 121.025819

In [39]:
# re-querying with a larger result set, to give more flexibility to discard

query_restaurants_iter2 = 'https://api.foursquare.com/v2/venues/explore?v=20190320&client_id={}&client_secret={}&ll={},{}&categoryId={}&limit=50&time=any&sortByPopularity=1'.format(
    Client_ID, Client_Secret, mkt_ctr_lat, mkt_ctr_long, restaurants_catID)

query_cafes_iter2 = 'https://api.foursquare.com/v2/venues/explore?v=20190320&client_id={}&client_secret={}&ll={},{}&categoryId={}&limit=50&time=any&sortByPopularity=1'.format(
    Client_ID, Client_Secret, mkt_ctr_lat, mkt_ctr_long, coffeeshop_catID)

In [40]:
iter2_restaurants_raw = requests.get(query_restaurants_iter2).json()
iter2_restaurants_raw

{'meta': {'code': 200, 'requestId': '5e8451b50f5968002662c37c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 989,
  'headerLocation': 'Forbes Park',
  'headerFullLocation': 'Forbes Park, Makati City',
  'headerLocationGranularity': 'neighborhood',
  'query': 'food',
  'totalResults': 52,
  'suggestedBounds': {'ne': {'lat': 14.561303908047854,
    'lng': 121.04394448633288},
   'sw': {'lat': 14.54409440238117, 'lng': 121.02450226255557}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5a664a5126659b35d9511631',
       'name': 'Marugame Udon',
       'location': {'address': '3/F Glorietta 4, Courtyard Dr, Ayala Center',
        'lat': 14.550808888423173,
        'lng': 121.02624

In [43]:
results2_restaurants = iter2_restaurants_raw['response']['groups'][0]['items']
results2_restaurants2 = json_normalize(results2_restaurants)
df_restaurants2 = results2_restaurants2[['venue.name','venue.location.state', 'venue.location.lat', 'venue.location.lng', 'venue.location.address', 'venue.id']]
df_restaurants2.rename(columns={"venue.name":"venue","venue.location.state":"city","venue.location.lat":"location_lat", "venue.location.lng":"location_lng", "venue.location.address":"address", "venue.id":"venue_id"},inplace=True)
df_restaurants2.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,venue,city,location_lat,location_lng,address,venue_id
0,Marugame Udon,Makati City,14.550809,121.026249,"3/F Glorietta 4, Courtyard Dr, Ayala Center",5a664a5126659b35d9511631
1,Tim Ho Wan,Makati City,14.552159,121.025458,G/F Glorietta 3,54647f5d498e9169e65819ce
2,Outback Steakhouse,Makati City,14.550850,121.026515,"Ground Flr, Glorietta 4",4b7b1c2cf964a520f7512fe3
3,McDonald's,Makati City,14.555526,121.034634,"G/F, SM Cyber Two, Sen. Gil Puyat Ave",4c200aed920076b07abac5e9
4,Hokkaido Ramen Santouka,Makati City,14.550445,121.026626,"Ground Flr, Glorietta 4",5125a7dee4b06c4475697e77


In [55]:
df_restaurants_nonGlorietta = df_restaurants2[((df_restaurants2['location_lat'] > 14.554477) | (df_restaurants2['location_lat'] < 14.549851)) & ((df_restaurants2['location_lng'] > 121.027825) | (df_restaurants2['location_lat'] < 121.022917))]
df_restaurants_nonGlorietta

,venue,city,location_lat,location_lng,address,venue_id
3,McDonald's,Makati City,14.555526,121.034634,"G/F, SM Cyber Two, Sen. Gil Puyat Ave",4c200aed920076b07abac5e9
9,Va Bene Pasta Deli,Makati City,14.547686,121.026627,"2nd Flr, Petron Service Station",4c2f09ea213c2d7fb933305d
10,JPMorgan Chase Cafeteria,Makati City,14.554787,121.043061,"7th Flr, NetPlaza Bldg, 31st St.",4c93ad3e58d4b60c39852129
11,Mitsuyado Sei-Men,Makati City,14.558172,121.033276,22 Jupiter St,5099e944e4b078b1ccc63c21
12,Maple,Makati City,14.547185,121.034806,"2nd Flr, San Antonio Plaza Arcade",50ca7218e4b0ed43555b8f1e
13,Lazy Bastard,Makati City,14.558248,121.033326,Basement Floor,5518cbb9498ec1c4b3ecf246
14,Pipino Vegetarian Food by Pino,Makati City,14.560522,121.031535,38 Jupiter Street Brgy. Bel Air,509c8881e4b051548cea09cd
16,Umu,Makati City,14.548781,121.026430,1223 EDSA,4bb85666cf2fc9b6c07b9f02
17,Jollibee,Makati City,14.555502,121.034902,Shell Edsa Buendia,5b98eba3d1a402002c88e639
18,Sweet Ecstasy,Makati City,14.557054,121.034271,Jupiter St,54182d17498e2d7b15d296c6


In [56]:
# dropping the office cafeteria in the list (domain knowledge)
df_restaurants_nonGlorietta.drop([10], inplace=True)

# resetting index
df_restaurants_nonGlorietta.reset_index(drop=True, inplace=True)

df_restaurants_nonGlorietta

,venue,city,location_lat,location_lng,address,venue_id
0,McDonald's,Makati City,14.555526,121.034634,"G/F, SM Cyber Two, Sen. Gil Puyat Ave",4c200aed920076b07abac5e9
1,Va Bene Pasta Deli,Makati City,14.547686,121.026627,"2nd Flr, Petron Service Station",4c2f09ea213c2d7fb933305d
2,Mitsuyado Sei-Men,Makati City,14.558172,121.033276,22 Jupiter St,5099e944e4b078b1ccc63c21
3,Maple,Makati City,14.547185,121.034806,"2nd Flr, San Antonio Plaza Arcade",50ca7218e4b0ed43555b8f1e
4,Lazy Bastard,Makati City,14.558248,121.033326,Basement Floor,5518cbb9498ec1c4b3ecf246
5,Pipino Vegetarian Food by Pino,Makati City,14.560522,121.031535,38 Jupiter Street Brgy. Bel Air,509c8881e4b051548cea09cd
6,Umu,Makati City,14.548781,121.026430,1223 EDSA,4bb85666cf2fc9b6c07b9f02
7,Jollibee,Makati City,14.555502,121.034902,Shell Edsa Buendia,5b98eba3d1a402002c88e639
8,Sweet Ecstasy,Makati City,14.557054,121.034271,Jupiter St,54182d17498e2d7b15d296c6
9,Basix,Makati City,14.549109,121.026150,Dusit Thani Manila,4d1ea3112eb1f04dafd7e1c1


In [57]:
# getting only the first 10 rows of remaining entries
df_restaurants_map2 = df_restaurants_nonGlorietta.iloc[0:10,:]
df_restaurants_map2

,venue,city,location_lat,location_lng,address,venue_id
0,McDonald's,Makati City,14.555526,121.034634,"G/F, SM Cyber Two, Sen. Gil Puyat Ave",4c200aed920076b07abac5e9
1,Va Bene Pasta Deli,Makati City,14.547686,121.026627,"2nd Flr, Petron Service Station",4c2f09ea213c2d7fb933305d
2,Mitsuyado Sei-Men,Makati City,14.558172,121.033276,22 Jupiter St,5099e944e4b078b1ccc63c21
3,Maple,Makati City,14.547185,121.034806,"2nd Flr, San Antonio Plaza Arcade",50ca7218e4b0ed43555b8f1e
4,Lazy Bastard,Makati City,14.558248,121.033326,Basement Floor,5518cbb9498ec1c4b3ecf246
5,Pipino Vegetarian Food by Pino,Makati City,14.560522,121.031535,38 Jupiter Street Brgy. Bel Air,509c8881e4b051548cea09cd
6,Umu,Makati City,14.548781,121.026430,1223 EDSA,4bb85666cf2fc9b6c07b9f02
7,Jollibee,Makati City,14.555502,121.034902,Shell Edsa Buendia,5b98eba3d1a402002c88e639
8,Sweet Ecstasy,Makati City,14.557054,121.034271,Jupiter St,54182d17498e2d7b15d296c6
9,Basix,Makati City,14.549109,121.026150,Dusit Thani Manila,4d1ea3112eb1f04dafd7e1c1


In [58]:
iter2_cafes_raw = requests.get(query_cafes_iter2).json()
iter2_cafes_raw

{'meta': {'code': 200, 'requestId': '5e8459faaba297001b527adf'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 989,
  'headerLocation': 'Forbes Park',
  'headerFullLocation': 'Forbes Park, Makati City',
  'headerLocationGranularity': 'neighborhood',
  'query': 'coffee shop',
  'totalResults': 75,
  'suggestedBounds': {'ne': {'lat': 14.562970666607319,
    'lng': 121.045674031682},
   'sw': {'lat': 14.546610001246316, 'lng': 121.0239292575401}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '59f192c56f0aa252bb503462',
       'name': 'Tim Hortons',
       'location': {'address': 'G/F Glorietta 4, Courtyard Dr, Ayala Center',
        'crossStreet': 'btwn North & East Dr',
        '

In [59]:
results2_cafes = iter2_cafes_raw['response']['groups'][0]['items']
results2_cafes2 = json_normalize(results2_cafes)
df_cafes2 = results2_cafes2[['venue.name','venue.location.state', 'venue.location.lat', 'venue.location.lng', 'venue.location.address', 'venue.id']]
df_cafes2.rename(columns={"venue.name":"venue","venue.location.state":"city","venue.location.lat":"location_lat", "venue.location.lng":"location_lng", "venue.location.address":"address", "venue.id":"venue_id"},inplace=True)
df_cafes2.head()

,venue,city,location_lat,location_lng,address,venue_id
0,Tim Hortons,Makati City,14.551287,121.026474,"G/F Glorietta 4, Courtyard Dr, Ayala Center",59f192c56f0aa252bb503462
1,FRNK Milk Bar,Makati City,14.551634,121.024924,"G/F, Glorietta 3",5ce93982c4df1d002c9e425d
2,Starbucks,Makati City,14.552657,121.024986,"Ground Flr, 6750 Ayala Tower, 6750 Ayala Ave",4dd26268cc3f7f8c122e7e1e
3,Starbucks,Makati City,14.551298,121.026868,"4th Flr, Glorietta 4",4b63d707f964a520c5932ae3
4,Myron's,Makati City,14.551258,121.026636,Courtyard Dr Fl 6,580ff4db38fad54ea66362ee


In [60]:
df_cafes_nonGlorietta = df_cafes2[((df_cafes2['location_lat'] > 14.554477) | (df_cafes2['location_lat'] < 14.549851)) & ((df_cafes2['location_lng'] > 121.027825) | (df_cafes2['location_lat'] < 121.022917))]
df_cafes_nonGlorietta

,venue,city,location_lat,location_lng,address,venue_id
10,The Pantry,Makati City,14.548744,121.025739,Ayala Centre,55fcdf7d498e81836aa8199b
11,Starbucks,Makati City,14.547354,121.034877,"2/F San Antonio Plaza Arcade, McKinley Rd, For...",4bd110b99854d13a651bf94d
15,Starbucks,Makati City,14.557225,121.033919,"G/F Rustan Coffee Bldg, 14 Jupiter St",57fc4c42498e5f31f1ae22ae
16,Starbucks,Makati City,14.557838,121.026491,"2/F Zuellig Bldg, Paseo de Roxas Ave",51a451ed498e5fd308c76d91
22,Starbucks Coffee,Makati City,14.549471,121.026880,3/F SM Makati,5cf511bff1936e002c4127cf
25,Krispy Kreme,Makati City,14.549625,121.026879,2/F SM Makati,5658099d498e91d786310e08
28,Happy Garden Cafe,Makati City,14.560936,121.030475,56 Jupiter Street,56a1c8e8498ee6f65f014fc9
31,Crumbsis,Makati City,14.558305,121.041264,NaN,592959b5db1d8113076093a7
34,UCC Park Café Express,Makati City,14.547619,121.026424,Petron Service Station,4d1098e0a3d9721e06e4d5fd
35,Bo's Coffee,Makati City,14.562227,121.028314,"Jupiter St, Brgy Bel-Air",4fdbcb28e4b056d3bcd4b918


In [61]:
# resetting index
df_cafes_nonGlorietta.reset_index(drop=True, inplace=True)

df_cafes_nonGlorietta

,venue,city,location_lat,location_lng,address,venue_id
0,The Pantry,Makati City,14.548744,121.025739,Ayala Centre,55fcdf7d498e81836aa8199b
1,Starbucks,Makati City,14.547354,121.034877,"2/F San Antonio Plaza Arcade, McKinley Rd, For...",4bd110b99854d13a651bf94d
2,Starbucks,Makati City,14.557225,121.033919,"G/F Rustan Coffee Bldg, 14 Jupiter St",57fc4c42498e5f31f1ae22ae
3,Starbucks,Makati City,14.557838,121.026491,"2/F Zuellig Bldg, Paseo de Roxas Ave",51a451ed498e5fd308c76d91
4,Starbucks Coffee,Makati City,14.549471,121.026880,3/F SM Makati,5cf511bff1936e002c4127cf
5,Krispy Kreme,Makati City,14.549625,121.026879,2/F SM Makati,5658099d498e91d786310e08
6,Happy Garden Cafe,Makati City,14.560936,121.030475,56 Jupiter Street,56a1c8e8498ee6f65f014fc9
7,Crumbsis,Makati City,14.558305,121.041264,NaN,592959b5db1d8113076093a7
8,UCC Park Café Express,Makati City,14.547619,121.026424,Petron Service Station,4d1098e0a3d9721e06e4d5fd
9,Bo's Coffee,Makati City,14.562227,121.028314,"Jupiter St, Brgy Bel-Air",4fdbcb28e4b056d3bcd4b918


In [62]:
# getting only the first 10 rows of remaining entries
df_cafes_map2 = df_cafes_nonGlorietta.iloc[0:10,:]
df_cafes_map2

,venue,city,location_lat,location_lng,address,venue_id
0,The Pantry,Makati City,14.548744,121.025739,Ayala Centre,55fcdf7d498e81836aa8199b
1,Starbucks,Makati City,14.547354,121.034877,"2/F San Antonio Plaza Arcade, McKinley Rd, For...",4bd110b99854d13a651bf94d
2,Starbucks,Makati City,14.557225,121.033919,"G/F Rustan Coffee Bldg, 14 Jupiter St",57fc4c42498e5f31f1ae22ae
3,Starbucks,Makati City,14.557838,121.026491,"2/F Zuellig Bldg, Paseo de Roxas Ave",51a451ed498e5fd308c76d91
4,Starbucks Coffee,Makati City,14.549471,121.026880,3/F SM Makati,5cf511bff1936e002c4127cf
5,Krispy Kreme,Makati City,14.549625,121.026879,2/F SM Makati,5658099d498e91d786310e08
6,Happy Garden Cafe,Makati City,14.560936,121.030475,56 Jupiter Street,56a1c8e8498ee6f65f014fc9
7,Crumbsis,Makati City,14.558305,121.041264,NaN,592959b5db1d8113076093a7
8,UCC Park Café Express,Makati City,14.547619,121.026424,Petron Service Station,4d1098e0a3d9721e06e4d5fd
9,Bo's Coffee,Makati City,14.562227,121.028314,"Jupiter St, Brgy Bel-Air",4fdbcb28e4b056d3bcd4b918


In [67]:
# re-starting from a blank map

mkt_map2 = folium.Map(location=[mkt_ctr_lat, mkt_ctr_long], zoom_start=16)

# add a red circle to map the geometric center
folium.Circle(
    [mkt_ctr_lat, mkt_ctr_long],
    radius=50,
    color='red',
    popup='Geometric Center',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6).add_to(mkt_map2)

# adding restaurants (updated data set) markers and proximity circle
for lat, lng, label in zip(df_restaurants_map2.location_lat, df_restaurants_map2.location_lng, df_restaurants_map2.venue):
    folium.Circle(
        [lat, lng],
        radius=walk_radius_meters,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.1
    ).add_to(mkt_map2)

# adding cafes markers and proximity circle
for lat, lng, label in zip(df_cafes_map2.location_lat, df_cafes_map2.location_lng, df_cafes_map2.venue):
    folium.Circle(
        [lat, lng],
        radius=walk_radius_meters,
        color='yellow',
        popup=label,
        fill = True,
        fill_color='yellow',
        fill_opacity=0.1
    ).add_to(mkt_map2)

# adding bars (unchanged data set) markers and proximity circle
for lat, lng, label in zip(df_bars_map.location_lat, df_bars_map.location_lng, df_bars_map.venue_name):
    folium.Circle(
        [lat, lng],
        radius=walk_radius_meters,
        color='orange',
        popup=label,
        fill = True,
        fill_color='orange',
        fill_opacity=0.1
    ).add_to(mkt_map2)

mkt_map2